In [1]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.NODE_Transformer_Dataset import NODE_Transformer_Dataset

from Model.SelfAttention import Self_attention, Attention_LSTM
from Model.LSTM import LSTM
from Model.NODE import NODE

import matplotlib.pyplot as plt

In [2]:
# 데이터 다운 & 전처리

transaction_df = pd.read_excel('../데이터/transaction/transaction_final.xlsx',index_col=0)
economy_df = pd.read_excel('../데이터/economy/economy_all.xlsx')['국고채금리']

In [28]:
# len_df = len(transaction_df)
# train_len = int(len_df *0.7)
# val_len = int(len_df * 0.3)

batch_size = 4
train_dataset = NODE_Transformer_Dataset(transaction_df[:300], economy_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
# val_dataset = RNN_Transformer_Dataset(df[train_len:train_len+val_len])
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

c:\Users\hkyoo\OneDrive\바탕 화면\SCI\코드\Dataset\NODE_Transformer_Dataset.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_data.drop_duplicates(subset=['시군구', '단지명', '계약년월'], keep='first', inplace=True) # 왜 rnn_transformer는 가능했지?


In [29]:
for x,y,z,w,a,b in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  print("Z 크기 : {}".format(z.shape))
  print("W 크기 : {}".format(w.shape))
  print("A 크기 : {}".format(a.shape))
  print("B 크기 : {}".format(b.shape))
  break

X 크기 : torch.Size([4, 158, 5])
Y 크기 : torch.Size([4, 158, 5])
Z 크기 : torch.Size([4, 1, 5])
W 크기 : torch.Size([4, 158, 1])
A 크기 : torch.Size([4, 158, 1])
B 크기 : torch.Size([4, 1, 1])


In [30]:
max_size = 158
window_size = 5
hidden_size =256

In [32]:
# gpu 연결
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# checkpoint 받기
eco_lstm = LSTM(window_size,256).to(device)
eco_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_economy_model.pth"))

tran_NODE = NODE(output_dim=1,hidden_dim=256,latent_dim=64).to(device)
# tran_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_transaction_model.pth"))

model = Self_attention(max_size, hidden_size, 1).to(device)

dummy = torch.rand((batch_size,max_size,hidden_size)).to(device)
model(dummy).shape

cpu is available


torch.Size([4, 158, 1])

In [33]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [34]:
tran_NODE(one_tran_input, one_time)[1][-1].shape

torch.Size([4, 256])

In [35]:
one_NODE.shape

torch.Size([4, 256])

### 학습하기

In [36]:
train_n = len(train_dataset)
# val_n = len(val_dataset)

# val_loss_list = []
best_train_loss = float('inf')
stop_counter = 0
stop_criteria = 3
max_len = max_size

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for tran_inputs, t_inputs, eco_inputs, tran_labels, t_label, eco_label in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        tran_inputs = tran_inputs.to(device) 
        t_inputs = t_inputs.to(device)  # batch * size * window_size
        eco_inputs = eco_inputs.to(device)  # batch*1*window_size
        tran_labels = tran_labels.to(device)  # batch_size*max_size*1
        t_label = t_label.to(device)   # batch * size * 1
        eco_label = eco_label.to(device) 
        t = torch.cat((t_inputs,t_label),dim=2)  # batch * size * window_size+1 (batch*158*6)
        
        # transaction hidden_state + economy hidden_state
        # batch 단위로 한번에 더함
        # 콜금리 사용
        eco_tensor = eco_lstm(eco_inputs)[1][0].squeeze().unsqueeze(1).to(device)   # batch * 1 * hidden_state
        eco_tensor = eco_tensor.repeat(1,max_len,1)  # batch * 158 * hidden
        tran_tensor = torch.zeros((batch_size,max_len,hidden_size)).to(device)
        for i in range(max_len):
            # batch * window -> window * batch * 1 
            one_tran_input = tran_inputs[:,i,:].permute(1,0).unsqueeze(2)
            # batch * window+1 -> window+1 * batch * 1
            one_time = t[:,i,:].permute(1,0).unsqueeze(2)
            # 1 * batch * hidden
            one_NODE = tran_NODE(one_tran_input, one_time)[1][-1]
            tran_tensor[:,i,:] = one_NODE
            
        emb_vector = tran_tensor + eco_tensor
        
        # 부동산 데이터가 0인 행은 embedding_vector도 0으로 전환  
        iszero_vector = torch.where(tran_labels.squeeze() == 0, tran_labels.squeeze(), 1.0)
           
        # 차원에 맞게 변경 후 계산 
        emb_vector = emb_vector.mul(iszero_vector.unsqueeze(2).repeat(1,1,hidden_size))
        
        #####################################
        # transformer 적용
        # loss 값은 부동산 가격이 0이 아닌 것에만 적용해서 학습
        out = model(emb_vector) # batch_size*max_size*1
        
        # out 값중 필요 없는 값은 0으로 변경
        out = out.mul(iszero_vector.unsqueeze(2))   
        
        labels = tran_labels.mul(iszero_vector.unsqueeze(2))
        
        # x값이 0이면 y값도 0인 상태!!!에서 아래 진행
        loss_all = 0
        for i in range(out.shape[1]):
            loss = criterion(out[:,i,:], labels[:,i,:])
            loss_all += loss
        loss_all = loss_all/iszero_vector.sum()   # 평균치만큼 업데이트!!!!!!!!
        
        mean_loss = loss_all/out.shape[1]
        optimizer.zero_grad() #
        mean_loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss_all.item() # 한 배치의 loss 더해주고,

        if mean_loss < best_train_loss:
            best_train_loss = mean_loss
            torch.save(model.state_dict(), '../데이터/checkpoint/best_rnn_transformer_model.pth')

    print(running_loss/train_n)

    # with torch.no_grad():
    #         print("Calculating validation results...")
    #         model.eval()
    #         val_loss = 0
    #         for inputs, labels in val_loader:
    #             price, economy = inputs
    #             price = price.to(device) # batch_size*max_size*256  (max_size = 최대 단지 개수)
    #             economy = economy.to(device)
    #             labels = labels.to(device) # batch_size*max_size*1
                
    #             # labels 가 0이 아니면 lstm 넣기
    #             embed = torch.zeros((batch_size,max_size,hidden_size))
    #             for i in range(labels.shape[0]):
    #                 for j in range(labels.shape[1]):
    #                     if labels[i][j] != 0:
    #                         embed[i][j] = lstm_price(price[i][j]) 
                            
    #             outs = model(inputs)

    #             loss_all = 0
    #             for i in range(out.shape[1]):
    #                 loss = criterion(out[:,i,:], labels[:,i,:])
    #                 loss_all += loss

    #         avg_val_loss = val_loss/val_n/out.shape[1]
    #         val_loss_list.append(avg_val_loss)

    #         if avg_val_loss < best_val_loss:
    #             best_val_loss = avg_val_loss
    #             stop_counter = 0            
    #             torch.save(model.state_dict(), 'Checkpoint/economic_best_model.pth')
    #         else:
    #             stop_counter += 1

    #         print("val")
    #         print(
    #             f"loss: {avg_val_loss:4.8} || "
    #         )

    #         if stop_counter >= stop_criteria:
    #             print('Early stopping triggered.')
    #             break
    
    print('======================================================')

# plt.figure(figsize=(10, 5))
# plt.plot(val_loss_list, label='Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

epoch : 0
train


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [6, 4, 1]] is at version 316; expected version 314 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).